In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The primary reason to apply regularization techniques is to prevent overfitting in machine learning models, particularly in complex models like neural networks. Overfitting occurs when a model becomes too tailored to the training data, capturing noise and random fluctuations rather than general patterns. As a result, the model's performance on unseen data (i.e., the test data) degrades, leading to poor generalization.

Regularization techniques aim to address overfitting by adding additional constraints to the model during training. These constraints encourage the model to focus on learning the essential patterns in the data while discouraging it from fitting noise or irrelevant details. By doing so, regularization improves the model's ability to generalize well to new, unseen data, which is the ultimate goal in machine learning.

Weight Decay Regularization:
Weight decay, also known as L2 regularization, is a technique used to prevent overfitting in neural networks by penalizing large weights. In weight decay, an additional term is added to the loss function that depends on the magnitude of the weights. The regularization term is proportional to the squared sum of all weights in the network.

The regularization term is defined as: Regularization_term = 

where 
 is the regularization strength, and 
 represents the weights of the network.

The role of weight decay in preventing overfitting is to discourage the model from relying too much on any particular feature or weight. By penalizing large weights, weight decay encourages the network to use smaller weights, resulting in a simpler model with reduced complexity. This, in turn, helps the model generalize better to unseen data and reduces the risk of overfitting.

To apply weight decay to a neural network during training, you need to add the regularization term to the loss function and adjust the learning process accordingly.

Dropout Regularization:
Dropout is a regularization technique that addresses overfitting by randomly deactivating (i.e., setting to zero) a fraction of neurons during training. In each training iteration, individual neurons are dropped with a certain probability, while during testing or inference, all neurons are active. This process effectively prevents complex co-adaptations among neurons, reducing the model's reliance on specific neurons and enhancing its generalization ability.

The role of dropout in preventing overfitting is to create an ensemble effect during training, where different subsets of neurons are activated or deactivated in each iteration. This ensemble approach leads to a more robust model that generalizes better to unseen data.

To apply dropout to a neural network is straightforward. You can add dropout layers after the activation functions in the hidden layers of your neural network.

Import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

Load the CIFAR-10 dataset and preprocess the data.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Define a neural network model with dropout layers.

In [ ]:
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
"""

In [ ]:
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
"""

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
net = Net()

Define the loss function (cross-entropy) and optimizer (Stochastic Gradient Descent with weight decay).

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.04, weight_decay=1e-3)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



Train the model with weight decay and dropout for a certain number of epochs.

In [ ]:
for epoch in range(5):  # Change the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print(f"[{epoch + 1}, {i + 1}] Loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training")

Evaluate the model's accuracy on the test set.

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {(100 * correct / total):.2f}%")

## NET 1
### Net 1 with 5 epochs
optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-4)
Accuracy on the test set: 45.23%
### Net 1 with 5 epochs with adam 
optim.Adam(net.parameters(), lr=0.04, weight_decay=1e-3)
Accuracy on the test set: 10.00%

### Net 1 with 5 epochs 
optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
Accuracy on the test set: 44.31%

## NET 2
### Net 2 with 5 epochs
optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-4)
Accuracy on the test set: 45.23%
### Net 2 with 5 epochs with adam 
optim.Adam(net.parameters(), lr=0.04, weight_decay=1e-3)
Accuracy on the test set: 10.00%

### Net 2 with 5 epochs 
optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
Accuracy on the test set: 52.09%

## NET 3
### Net 3 with 5 epochs
optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-4)
Accuracy on the test set: 53.91%

### Net 3 with 5 epochs with adam 
optim.Adam(net.parameters(), lr=0.04, weight_decay=1e-3)
Accuracy on the test set: 10.00%

### Net 3 with 5 epochs 
optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
Accuracy on the test set: 55.13%
Accuracy on the test set: 52.09%

Show an example of an image from the test dataset and its predicted class using the trained regularized model.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # Unnormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

for data in testloader:
    images, labels = data
    imshow(torchvision.utils.make_grid(images))  # Show the image
    print('True Label: ', labels)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    print('Predicted Label: ', predicted)
    break